# Load Data

California housing data fetcher is broken -- using Boston housing data instead

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import normalize
from sklearn.datasets import load_boston

housing = load_boston()
m, n = housing.data.shape  # (m = # of instances; n = # of instances)
#print('shape = ({},{})'.format(m, n))
# add a bias feature (necessary for lin reg)
housing_biased = np.c_[np.ones((m,1)), housing.data]
#print('shape = ', housing_biased.shape)

# **Normal Equation**

Computes parameter vector, **$\theta$**, in one go

$\hat\theta = \left(X^T\cdot X\right)^{-1}\cdot X^T\cdot y $

In [3]:
X = tf.constant(housing_biased, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
# theta = inv(XT * X) * XT * y
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_val = theta.eval()
    print(theta_val)

[[  3.65391960e+01]
 [ -1.07211098e-01]
 [  4.63852249e-02]
 [  2.08310299e-02]
 [  2.68740368e+00]
 [ -1.78212509e+01]
 [  3.80236006e+00]
 [  7.74049258e-04]
 [ -1.47647619e+00]
 [  3.05743963e-01]
 [ -1.23250699e-02]
 [ -9.54248607e-01]
 [  9.38500185e-03]
 [ -5.25576472e-01]]


# **Batch Gradient Descent (BGD)**

Find model parameters incrementally, starting with random values for **$\theta$**. At each step, calculate the gradient of the cost function with respect to each model parameter, $\theta_j$, denoted

$\frac{\partial}{\partial \theta_j}MSE(\mathbf{\theta}) = \frac 2m \sum_{i=1}^{m} \left(\mathbf{\theta}^T \cdot \mathbf{x}^{(i)} - y^{(i)}\right)x_j^{(i)}$

Instead of computing each partial derivative individually, we compute them in one go to find the gradient vector, $\nabla_\theta MSE(\theta)$ using the following equation:

$\nabla_\theta MSE(\theta) = \begin{pmatrix} \frac{\partial}{\partial \theta_0}MSE(\theta) \\ \frac{\partial}{\partial \theta_1}MSE(\theta) \\ \vdots \\ \frac{\partial}{\partial \theta_n}MSE(\theta) \end{pmatrix} = \frac2m \mathbf{X}^T\cdot (\mathbf{X} \cdot \theta - \mathbf{y}) $

The gradient vector points "uphill", so we negate it since we are minimizing the cost function. We also have a learning rate hyperparameter, $\eta$, used to scale the gradient vector at each learning step. So, our equation for each step is:

$\theta^{(next)} = \theta - \eta\nabla_{\theta}MSE(\theta) $

In the next cell, the gradient is computed manually:

In [15]:
eta = 0.01 # learning rate 
n_epochs = 1000

housing_biased_norm = normalize(housing_biased)
X = tf.constant(housing_biased_norm, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# manually compute gradients
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - eta * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        if epoch == n_epochs-1:
            print("Final MSE = ", mse.eval())
        sess.run(training_op)
    theta_best = theta.eval()
    print(theta_best)

Epoch 0 MSE =  622.075
Epoch 100 MSE =  89.3275
Epoch 200 MSE =  74.5117
Epoch 300 MSE =  72.305
Epoch 400 MSE =  70.8024
Epoch 500 MSE =  69.6332
Epoch 600 MSE =  68.7169
Epoch 700 MSE =  67.9959
Epoch 800 MSE =  67.426
Epoch 900 MSE =  66.9729
Final MSE =  66.6135
[[  0.2783739 ]
 [ -0.23561998]
 [  2.33036089]
 [  0.82568747]
 [ -0.28459436]
 [ -0.58693224]
 [  1.41693676]
 [  2.02212644]
 [ -0.54973865]
 [  0.49315774]
 [  9.5137291 ]
 [ -0.12049896]
 [ 23.73558617]
 [ -0.74865985]]


# BGD using autodiff

tf.gradients() function takes in an op (mse) and a list of variables (just theta in this case), and creates a list of ops (one per variable) to compute the gradients of the op with respect to each variable. The gradients node computes the gradient vector of MSE w.r.t. theta

The end result is more or less the same as using the manually computed gradient, but this example uses a *very* simple cost function.

In [35]:
eta = 0.01 # learning rate 
n_epochs = 1000

housing_biased_norm = normalize(housing_biased)
X = tf.constant(housing_biased_norm, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# This is where we use autodiff
gradients = tf.gradients(mse, [theta])[0] 
training_op = tf.assign(theta, theta - eta * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        if epoch == n_epochs-1:
            print("Final MSE = ", mse.eval())
        sess.run(training_op)
    theta_best = theta.eval() 
    print(theta_best)

Epoch 0 MSE =  591.538
Epoch 100 MSE =  86.8639
Epoch 200 MSE =  73.0164
Epoch 300 MSE =  71.0756
Epoch 400 MSE =  69.7683
Epoch 500 MSE =  68.7508
Epoch 600 MSE =  67.9528
Epoch 700 MSE =  67.3242
Epoch 800 MSE =  66.8268
Epoch 900 MSE =  66.4308
Final MSE =  66.1161
[[ -0.39421716]
 [  0.40667704]
 [  3.96623063]
 [  0.27636206]
 [ -0.17432758]
 [  0.70810735]
 [  0.15927659]
 [  1.14151692]
 [ -0.20456119]
 [ -0.48585832]
 [  9.48361588]
 [  1.5889343 ]
 [ 23.84345436]
 [ -1.0559268 ]]


# Mini-Batch GD

Instead of computing the gradients using the entire training set, we use a small, random subset of instances called *mini-batches*. This provides a large performance speedup at the cost of accuracy. To accomplish this, we need to replace X and y at every iteration with a different mini-batch. Placeholder nodes accomplish this by outputting data at runtime, instead of performing some computation. (If no value is specified for a placeholder, you get an exception)

When evaluating a node dependent on a placeholder, we pass in a feed_dict to the eval() method.

In [37]:
eta = 0.01 # learning rate 
n_epochs = 1000

housing_biased_norm = normalize(housing_biased)
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
batch_size = 100
n_batches = int(np.ceil(m/batch_size))
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# This is where we use autodiff
gradients = tf.gradients(mse, [theta])[0] 
training_op = tf.assign(theta, theta - eta * gradients)

init = tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    start_ind = batch_index * batch_size
    end_ind = start_ind + batch_size
    if end_ind > m: end_ind = m # cannot go out of bounds
    #print('({},{})'.format(start_ind, end_ind))
    X_batch = housing_biased_norm.data[start_ind:end_ind]
    y_batch = housing.target.reshape(-1,1)[start_ind:end_ind]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
        if epoch == n_epochs-1:
            print("Final MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
    theta_best = theta.eval() # no need for feed_dict, since theta does not depend on X or y
    print(theta_best)

Epoch 0 MSE =  325.135
Epoch 100 MSE =  29.1659
Epoch 200 MSE =  31.6684
Epoch 300 MSE =  32.8169
Epoch 400 MSE =  33.1949
Epoch 500 MSE =  33.1788
Epoch 600 MSE =  32.9741
Epoch 700 MSE =  32.6856
Epoch 800 MSE =  32.3656
Epoch 900 MSE =  32.0389
Final MSE =  31.7206
[[ -0.30211103]
 [ -1.19279504]
 [ 12.65120602]
 [ -2.79090834]
 [ -0.23257811]
 [ -0.76165146]
 [  2.00952196]
 [ -4.61977196]
 [ -0.0421115 ]
 [  0.58151853]
 [  8.58374977]
 [ -0.16875364]
 [ 24.57528496]
 [ -5.29968739]]
